<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/WIP/Tutorial_for_Neural_Network_Verification_Fairness_with_Marabou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verification Of Fairness With Marabou

As mentioned in this subsection we guide you through the verification of the fairness property with marabou. For that we will train a network on another dataset and then verify the fairness with marabou as we did with robustness.

## Training And Exporting The Neural Network


We train the neural network on the [Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/census+income). 
This classification problem has a set of inputs like age, gender, workclass and other. 
The classifier now has to predict whether the income is above or below \$50,000.

---

Listing of input features (encoded with integers starting with 1):

* age: continuous.

* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

* fnlwgt: continuous.

* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

* education-num: continuous.

* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

* sex: Female, Male.

* capital-gain: continuous.

* capital-loss: continuous.

* hours-per-week: continuous.

* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

Output (encoded with two neurons): 
* <=50K or >50K

To check the fairness in Marabou, we need to slightly modify our neural network

In [ ]:
%%capture
census_train_dataloader, census_test_dataloader = mnist_trainer.load_census_income_dataset(batch_size=64)

In [ ]:
# train model
census_model = mnist_trainer.train_model(
    NeuralNetwork(input_dim=14, output_dim=2, number_of_neurons=20), 
    epochs=5, 
    train_dataloader=census_train_dataloader,    
    test_dataloader=census_test_dataloader,
    )

# export model
census_model_filename = "census_net.onnx"
export_as_onnx(census_model, model_filename=census_model_filename, dummy_input=torch.randn(2, 14))

## Verify Fairness With Marabou

As before we need to read the network with marabou again.

In [ ]:
census_network = Marabou.read_onnx(census_model_filename)

inputVars1 = census_network.inputVars[0][0]
inputVars2 = census_network.inputVars[0][1]
outputVars1 = census_network.outputVars[0][0]
outputVars2 = census_network.outputVars[0][1]

#inputVars1, inputVars2 = split_list(inputVars)
#outputVars1, outputVars2 = split_list(outputVars)

The fairness property states, that the neural network should output the same result, if exactly one input, but no other, is changed. This can be applied to sensitive attributes like sex or race. In our example a sensitive attribute can be the sex. 
So if the network predicts that a male earns more than \\$50,000 with given attributes, then a female with the same attributes also has to earn more than \\$50,000, when the network is fair.

Let $\mathcal{N}$ be our neural network with $\mathcal{f}_{\mathcal{N}}: \mathbb{R}^m\rightarrow\mathbb{R}^n$ and $i \in \{1...,m\}$ is a sensitive attribute. 

The neural network $\mathcal{N}$ is fair if:
$$
∀\vec{x},\vec{y}\in\mathbb{R}^m:(x_i \neq y_i \wedge \displaystyle\bigwedge_{j \neq i} x_j = y_j) \Rightarrow \mathcal{f}_{\mathcal{N}}(\vec{x}) = \mathcal{f}_{\mathcal{N}}(\vec{y}).
$$

In [ ]:
inputs, classes = next(iter(census_train_dataloader))
inputs = inputs[0].numpy()
correct_class = classes[0].item()
print(inputs)
print(correct_class)

network_ouput = census_network.evaluateWithoutMarabou([np.array([inputs, inputs])])[0]
predicted_class = np.argmax(network_ouput)
print(network_ouput)
print(predicted_class)

In [ ]:
# sex: index 9
i = 9

#network.addEquality([inputVars1[i], inputVars2[i]], [1, -1], 0) # 1*x + (-1)*y = 0

for x, y in zip(inputVars1, inputVars2):
  if x != inputVars1[i]:
    census_network.addEquality([x, y], [1, -1], 0) # 1*x + (-1)*y = 0

for i in range(outputVars1.shape[0]):
  print(f"iteration: {i}")
  census_network.maxList = []
  census_network.addMaxConstraint(set(outputVars1), outputVars1[i])
  census_network.addMaxConstraint(set(outputVars2), outputVars2[i])
  
  # solve
  exit_code, vals, stats = census_network.solve(verbose = False, options = options)
  # if solution found, break
  print(exit_code)
  if len(vals) > 0:
    break
